In [2]:
# creates a trade history by reading InteractiveBrokers' CSV-formated report

import pandas as pd
import numpy as np
import math
import Tools.data_cleanup_helpers as d_helpers
import Tools.helpers as helpers
import datetime

data = pd.read_csv('P:/aaa.csv', header = None)

def get_sheet(data, s):
    df = data[data[0]==s]
    return d_helpers.make_first_row_as_header(df)

def update_ticker(df, header = 'Ticker'):
    for i in df.index:
        currency = df.loc[i,'Currency']
        if currency == 'HKD':
            df.loc[i,header] += ' HK'
        elif currency == 'JPY':
            df.loc[i,header] = df.loc[i, header][:-2] + ' JP'
        elif currency == 'GBP':
            df.loc[i,header] += ' LN'

def str_to_date(s):
    a = helpers.string_to_date(s)
    return pd.Timestamp(year=a.year, month=a.month, day=a.day)

def weekday(d):
    if d.weekday() == 5:
        return d + datetime.timedelta(days=2)
    elif d.weekday() == 6:
        return d + datetime.timedelta(days=1)
    else:
        return d

In [29]:
trades = get_sheet(data, 'Trades')
trades = trades[trades['Header']=='Data'][['Asset Category','Currency','Symbol','Date/Time','Quantity','Proceeds','Comm/Fee']]


def stocks_to_equity(s):
    if s == 'Stocks':
        return 'Equity'
    else:
        return s


trades['Type'] = trades['Asset Category'].map(stocks_to_equity)
trades['Date'] = trades['Date/Time'].map(str_to_date)
trades['Comm/Fee'] = trades['Comm/Fee'].astype('float32')
trades['Amount'] = trades['Proceeds'].astype('float32') + trades['Comm/Fee']
trades['Quantity'] = trades['Quantity'].map(d_helpers.convert_number_if_applicable)
trades = trades[['Currency','Symbol','Quantity','Comm/Fee','Type','Date','Amount']]

def trx_type(k):
    if k>0:
        return 'Buy'
    else:
        return 'Sell'

trades['Trx type'] = trades['Quantity'].map(trx_type)    
a = pd.pivot_table(trades,index=['Currency','Symbol','Type','Date','Trx type'],values=['Quantity','Comm/Fee','Amount'], aggfunc=[np.sum])
a.columns = a.columns.get_level_values(1)
trades = a.reset_index()[['Trx type','Symbol','Date','Amount','Comm/Fee','Currency','Quantity','Type']]

for i in trades.index:
    t = trades.loc[i,'Type']
    if t == 'Equity':
        currency = trades.loc[i,'Currency']
        if currency == 'HKD':
            trades.loc[i,'Symbol'] += ' HK'
            trades.loc[i,'Date'] += datetime.timedelta(days=1)
        elif currency == 'JPY':
            trades.loc[i,'Symbol'] = trades.loc[i,'Symbol'][:-2] + ' JP'
            trades.loc[i,'Date'] += datetime.timedelta(days=1)
        elif currency == 'GBP':
            trades.loc[i,'Symbol']  += ' LN'

trades.rename(columns = {'Symbol':'Ticker'}, inplace = True)

def option_type(sss):
    if (sss[sss.find(' ')+1:]).find('P')>-1:
        return 'Put'
    if (sss[sss.find(' ')+1:]).find('C')>-1:
        return 'Call'
    return 'Other'

t = trades[trades['Type']=='Equity and Index Options']
tt = t['Ticker'].map(option_type)
trades.loc[tt.index,'Type'] = tt


def covert_option_name(s):
    l = s.split()
    ticker = l[0]
    d = datetime.datetime.strptime(l[1], '%d%b%y').strftime('%m/%d/%y')
    px = float(l[2])

    t = l[3]
    return ticker + ' ' + d + ' '+t+str(px)
tt = t['Ticker'].map(covert_option_name)
trades.loc[tt.index,'Ticker'] = tt

In [9]:
deposits = get_sheet(data, 'Deposits & Withdrawals')
deposits = deposits[deposits['Header']=='Data']
deposits = deposits[deposits['Currency']!='Total']
deposits['Trx type'] = 'Deposit'
deposits['Date'] = deposits['Settle Date'].map(str_to_date)
deposits['Amount'] = deposits['Amount'].map(d_helpers.convert_number_if_applicable)
deposits = deposits.reset_index()[['Trx type', 'Currency', 'Date', 'Amount']]

In [10]:
dividends = get_sheet(data, 'Dividends')
dividends = dividends[dividends['Header']=='Data']
dividends = dividends[dividends['Currency']!='Total']
dividends['Trx type'] = 'Dividends'
dividends = dividends[dividends['Date'].fillna('Ha') != 'Ha']
dividends['Amount'] = dividends['Amount'].map(d_helpers.convert_number_if_applicable)
dividends['Ticker'] = dividends['Description'].map(lambda s: s[0:s.find('(')])
dividends['Date'] = dividends['Date'].map(str_to_date)
dividends = dividends[['Trx type','Ticker','Date','Currency','Amount']]
update_ticker(dividends)

In [11]:
interest = get_sheet(data, 'Interest')
interest = interest[interest['Header']=='Data']
interest = interest[interest['Date'].map(lambda s: type(s) is str)]
interest['Date'] = interest['Date'].map(str_to_date)
interest['Amount'] = interest['Amount'].map(d_helpers.convert_number_if_applicable)
def type_of_interest(s):
    if s.find('Short Stock')>-1:
        return 'Short interest'
    else:
        return 'Other'

interest['Ticker'] = interest['Description'].map(type_of_interest)
interest.rename(columns = {'Interest':'Trx type'}, inplace=True)
interest = interest[['Trx type','Ticker','Date', 'Currency','Amount']]

In [30]:
output = pd.concat([trades, deposits, dividends, interest], axis=0, sort=False).sort_values(by=['Date'])
output = output.reset_index().drop(columns='index')

output['Date'] = output['Date'].map(weekday)
output.to_excel('P:/output_trades.xlsx')

In [164]:
def str_to_28th_of_month(s):
    a = helpers.string_to_date(s)
    return weekday(pd.Timestamp(year=a.year, month=a.month, day=28))

borrow_fee = get_sheet(data, 'Borrow Fee Details')
borrow_fee = borrow_fee[borrow_fee['Header']=='Data']
borrow_fee = borrow_fee[borrow_fee['Value Date'].map(lambda s: type(s) is str)]
borrow_fee['Date'] = borrow_fee['Value Date'].map(str_to_28th_of_month)
borrow_fee['Amount'] = borrow_fee['Borrow Fee'].map(d_helpers.convert_number_if_applicable)
borrow_fee.rename(columns = {'Symbol':'Ticker'}, inplace = True)
borrow_fee = borrow_fee[['Ticker','Date', 'Currency','Amount']]
update_ticker(borrow_fee)
borrow_fee = pd.pivot_table(borrow_fee,index=['Currency','Ticker', 'Date'],values=['Amount'], aggfunc=[np.sum])
borrow_fee.columns = borrow_fee.columns.get_level_values(1)
borrow_fee.reset_index(inplace = True)
borrow_fee['Trx type'] = 'Borrow Fee'

-785.2400000000001